In [3]:
from transformers import AutoTokenizer, AutoModel
import torch
import onnx
from pathlib import Path

# Load model and tokenizer
model_id = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# Example sentence for export shape reference
example_text = "Ez egy példa mondat."
inputs = tokenizer(example_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Export path
onnx_path = Path("../rknn_model/minilm_sentence_embed.onnx")

# Export to ONNX
torch.onnx.export(
    model,
    args=(inputs["input_ids"], inputs["attention_mask"]),
    f=onnx_path.as_posix(),
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state", "pooler_output"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence"},
        "attention_mask": {0: "batch_size", 1: "sequence"},
        "last_hidden_state": {0: "batch_size", 1: "sequence"},
        "pooler_output": {0: "batch_size"}
    },
    opset_version=14,
    do_constant_folding=True
)

# Validate ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)

onnx_path


WindowsPath('../rknn_model/minilm_sentence_embed.onnx')